In [0]:
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd

from os import scandir, getcwd
from prettytable import PrettyTable

#Cargo top ten

In [0]:
import csv
import networkx as nx
data={}
with open('/content/most_important_bw', newline='') as csvfile:
    data['bw'] = list(csv.reader(csvfile))
with open('/content/most_important_degree', newline='') as csvfile:
    data['dg'] = list(csv.reader(csvfile))
with open('/content/most_important_pr', newline='') as csvfile:
    data['pr'] = list(csv.reader(csvfile))


In [0]:
def ls(ruta = getcwd()):
    return [arch.name for arch in scandir(ruta) if arch.is_file()]

def load_networks(path):
    files = ls(path)
    graphs = {}
    full = nx.Graph()
    coauthors_full = set() 

    for file in files:
        name = file.split('_')[-1].split('.')[0]
        g = nx.read_weighted_edgelist(path+'/'+file, delimiter=',')        
        coauthors = set()
        full = nx.compose(full, g)
        with open(path+'/'+file) as f:
            lines = f.readlines()
            for line in lines:
                author = line.split(',')[0]
                coauthors.add(author)
                coauthors_full.add(author)
            g = g.subgraph(coauthors)            
        graphs[name] = g   
    graphs['full']=full.subgraph(list(coauthors_full))
    return graphs


In [0]:
def get_comunidad(node,communitys):
  result=set()
  for inst,g in communitys.items():
    if node in g:

      result.add(inst)
  return result



In [0]:
path ='/content/grafos'
graphs = load_networks(path)
full={}  
full['Google']=graphs['google']
full['MIT']=graphs['Massachusetts Institute of Technology']
full['Microsoft']=graphs['microsoft']
full['Facebook']=graphs['facebook']
full['Carnegie']=graphs['Carnegie Mellon University']
full['Illinois']=graphs['Univ']

G=nx.read_gml('/content/full.gml')



In [49]:
authors_connexion={}
neigborsh=[]
for mesure,authors in data.items():
  authors_connexion[mesure]={}
  for author in authors:
    author=author[0]
    insts = list(get_comunidad(author,full))
    count_same_inst=0
    for inst in insts:
      neigborsh=G.neighbors(author)
      for neigh in list(neigborsh):
        NI = get_comunidad(neigh,full)
        for ni in NI:
          if ni not in inst:
            if (author,inst,ni) in authors_connexion[mesure].keys():
              authors_connexion[mesure][(author,inst,ni)]+=1
            else:
              authors_connexion[mesure][(author,inst,ni)]=1
          else:
            count_same_inst+=1
labels=full.keys()

#Color
R = "\033[0;31;40m" #RED
G = "\033[0;32;40m" # GREEN
Y = "\033[0;33;40m" # Yellow
B = "\033[0;34;40m" # Blue
N = "\033[0m" # R

for mesure in authors_connexion.keys():
  row={}
  for key,val  in authors_connexion[mesure].items():
    table=[]
    for inst in list(labels):
      if key[2] in inst:
        table.append(int(val))
      else:
        table.append(0)
    if (key[0],key[1]) in row.keys():
      row[(key[0],key[1])].append(table)
    else:
      row[(key[0],key[1])]=[table]
  x = PrettyTable()
  x.field_names = ["Autor","Pertenece a","Medida",'Google', 'MIT', 'Microsoft', 'Facebook', 'Carnegie', 'Illinois','total']
  i=0
  for key, val in row.items():
    s = [sum(a) for a in zip(*val)]
    total=sum(s)
    x.add_row([key[0],key[1],G+mesure+N,*[sum(a) for a in zip(*val)],total])
  print("Autores importantes por Medida")
  print(x)





Autores importantes por Medida
+--------------------+-------------+--------+--------+-----+-----------+----------+----------+----------+-------+
|       Autor        | Pertenece a | Medida | Google | MIT | Microsoft | Facebook | Carnegie | Illinois | total |
+--------------------+-------------+--------+--------+-----+-----------+----------+----------+----------+-------+
|     Quoc_V._Le     |    Google   |   bw   |   0    |  0  |     1     |    1     |    2     |    0     |   4   |
|    Eric_P._Xing    |   Carnegie  |   bw   |   9    |  4  |     6     |    2     |    0     |    0     |   21  |
|   Samuel_Madden    |     MIT     |   bw   |   7    |  0  |     11    |    2     |    8     |    1     |   29  |
|      Wei_Wang      |  Microsoft  |   bw   |   21   |  3  |     0     |    2     |    0     |    0     |   26  |
| Christos_Faloutsos |   Carnegie  |   bw   |   9    |  2  |     1     |    2     |    0     |    3     |   17  |
|   Oriol_Vinyals    |    Google   |   bw   |   0    |  0